In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [2]:
#Data work
data = pd.read_csv("retail_data.csv")

data.dropna(axis=0, how='any', thresh=None, subset=None,inplace=True)
data['CustomerID'] = data['CustomerID'].astype(np.int64)

index = []
prod = []
c=0

for i in data["Product"]:
    if i not in prod:
        index.append(c)
        prod.append(i)
        c+=1
    else:
        index.append(prod.index(i))

lst = []
for i in data["Product"]:
    lst.append(1)

    
data.loc[:,'Action'] = lst
data.loc[:,'PRODUCT_ID'] = index


required_data = data[['PRODUCT_ID', 'Product', 'CustomerID', 'Action']]

/Users/viktortitarenko/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3043: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = yield from self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data

,Bill,MerchandiseID,Product,Quota,BillDate,Amount,CustomerID,Country,Action,PRODUCT_ID
0,532657,21314,Small glass heart trinket pot,12,2018-11-14,4.10,14562,United Kingdom,1,0
1,563214,22383,Lunch bag suki design,2,2019-8-14,3.65,16370,United Kingdom,1,1
2,507597,22561,Wooden school colouring set,12,2018-5-10,3.65,17700,United Kingdom,1,2
3,491634,21588,Retro spot giant tube matches,1,2017-12-11,4.55,17841,United Kingdom,1,3
4,496007,85232B,Set/3 russian doll stacking tins,3,2018-1-28,6.95,15203,United Kingdom,1,4
...,...,...,...,...,...,...,...,...,...,...
1067364,502588,22423,Regency cakestand 3 tier,1,2018-3-25,14.75,16550,United Kingdom,1,36
1067366,499973,21529,Retro spot ceramic toastrack,1,2018-3-3,5.45,13595,United Kingdom,1,2180
1067367,514387,21980,Pack of 12 red spotty tissues,48,2018-7-2,2.29,12633,Germany,1,2007
1067368,501897,22268,Easter decoration sitting bunny,10,2018-3-21,2.85,17539,United Kingdom,1,109


In [4]:
product_title = data[['PRODUCT_ID', 'Product']]
product_customer = data[['CustomerID', 'PRODUCT_ID', 'Action']]
product_title = product_title.drop_duplicates()
product_title.info()
product_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5299 entries, 0 to 1066472
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PRODUCT_ID  5299 non-null   int64 
 1   Product     5299 non-null   object
dtypes: int64(1), object(1)
memory usage: 124.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 824364 entries, 0 to 1067370
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   CustomerID  824364 non-null  int64
 1   PRODUCT_ID  824364 non-null  int64
 2   Action      824364 non-null  int64
dtypes: int64(3)
memory usage: 25.2 MB


In [5]:
num_customers = len(product_customer.CustomerID.unique())
num_products = len(product_customer.PRODUCT_ID.unique())
print('There are {} unique customers and {} unique products in this data'.format(num_customers, num_products))

There are 5942 unique customers and 5299 unique products in this data


In [6]:
#product-customer matrix
product_customer_matrix = product_customer.pivot_table(index='PRODUCT_ID', columns='CustomerID', values='Action').fillna(0)

#mapper from product name to idx
product_to_idx = {product: i for i, product in enumerate(list(product_title.set_index('PRODUCT_ID').loc[product_customer_matrix.index].Product))}

#scipy sparse matrix
product_customer_matrix_sparse = csr_matrix(product_customer_matrix.values)

In [7]:
#Model

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(product_customer_matrix_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [8]:
def make_recommendation(model_knn, data, mapper, idx, n_recommendations):
    model_knn.fit(data)
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    reverse_mapper = {v: k for k, v in mapper.items()}
    print('Recommendations :')
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}  --->  {2}'.format(i+1, reverse_mapper[idx], dist))

In [30]:
#Enter index of product to get recommendations
idx = 1111

make_recommendation(model_knn, product_customer_matrix_sparse, product_to_idx, idx, n_recommendations=10)

Recommendations :
1: Ribbon reel lace design   --->  0.7830522781022249
2: Embroidered ribbon reel claire  --->  0.6636681055272557
3: Embroidered ribbon reel ruby   --->  0.5791574715704937
4: Embroidered ribbon reel sophie    --->  0.5254385957398711
5: Embroidered ribbon reel rachel   --->  0.5140529058495635
6: Embroidered ribbon reel susie   --->  0.45697660003839546
7: Embroidered ribbon reel rebecca   --->  0.45669463200556737
8: Embroidered ribbon reel daisy   --->  0.43790261604539715
9: Embroidered ribbon reel rosie  --->  0.43356499769288726
10: Embroidered ribbon reel sally   --->  0.3893774811762918


In [10]:
product_title

,PRODUCT_ID,Product
0,0,Small glass heart trinket pot
1,1,Lunch bag suki design
2,2,Wooden school colouring set
3,3,Retro spot giant tube matches
4,4,Set/3 russian doll stacking tins
...,...,...
1049660,5294,Camoflage cushion cover
1057685,5295,S/2 hyacinth bulb candles
1062265,5296,S/4 heart crystal fridge magnets
1062389,5297,Painted heart wreath with bell
